In [1]:
import tensorflow.compat.v1.keras
from tensorflow.compat.v1.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.compat.v1.keras.preprocessing import image
import requests
from skimage.segmentation import slic
import matplotlib.pylab as pl
import numpy as np
import shap

# load model data
r = requests.get('https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json')
feature_names = r.json()
model = VGG16(weights='imagenet', include_top=True)
X,y = shap.datasets.imagenet50()
# load an image
img = X[[41]][0]
img_orig = X[[41]]





In [2]:
# segment the image so with don't have to explain every pixel
segments_slic = slic(img, n_segments=50, compactness=30, sigma=3)

skimage.measure.label's indexing starts from 0. In future version it will start from 1. To disable this warning, explicitely set the `start_label` parameter to 1.


In [3]:
# define a function that depends on a binary mask representing if an image region is hidden
def mask_image(zs, segmentation, image, background=None):
    if background is None:
        background = image.mean((0,1))
    out = np.zeros((zs.shape[0], image.shape[0], image.shape[1], image.shape[2]))
    for i in range(zs.shape[0]):
        out[i,:,:,:] = image
        for j in range(zs.shape[1]):
            if zs[i,j] == 0:
                out[i][segmentation == j,:] = background
    return out
def f(z):
    return model.predict(preprocess_input(mask_image(z, segments_slic, img_orig, 255)))

In [4]:
# use Kernel SHAP to explain the network's predictions, 50 due to 50 segments
explainer = shap.KernelExplainer(f, np.zeros((1,50)))
shap_values = explainer.shap_values(np.ones((1,50)), nsamples=1000) # runs VGG16 1000 times

Provided model function fails when applied to the provided data set.


ValueError: could not broadcast input array from shape (224,224,3) into shape (1,224,224)